# Loading in data

## Data Section 1 - The Basics

**This section provides everything you need to build a simple prediction model and submit predictions.**

- Team ID's and Team Names
- Tournament seeds since 1984-85 season
- Final scores of all regular season, conference tournament, and NCAA® tournament games since 1984-85 season
- Season-level details including dates and region names
- Example submission file for stage 1
  
By convention, when we identify a particular season, we will reference the year that the season ends in, not the year that it starts in.

## Data Section 2 - Team Box Scores

**This section provides game-by-game stats at a team level (free throws attempted, defensive rebounds, turnovers, etc.) for all regular season, conference tournament, and NCAA® tournament games since the 2003 season (men) or since the 2010 season (women).**

Team Box Scores are provided in "Detailed Results" files rather than "Compact Results" files. However, the two files are strongly related.

In a Detailed Results file, the first eight columns (**Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, and NumOT**) are exactly the same as a Compact Results file. However, in a Detailed Results file, there are many additional columns. The column names should be self-explanatory to basketball fans (as above, "W" or "L" refers to the winning or losing team):

- WFGM - field goals made (by the winning team)
- WFGA - field goals attempted (by the winning team)
- WFGM3 - three pointers made (by the winning team)
- WFGA3 - three pointers attempted (by the winning team)
- WFTM - free throws made (by the winning team)
- WFTA - free throws attempted (by the winning team)
- WOR - offensive rebounds (pulled by the winning team)
- WDR - defensive rebounds (pulled by the winning team)
- WAst - assists (by the winning team)
- WTO - turnovers committed (by the winning team)
- WStl - steals (accomplished by the winning team)
- WBlk - blocks (accomplished by the winning team)
- WPF - personal fouls committed (by the winning team)
- 
(and then the same set of stats from the perspective of the losing team: **LFGM** is the number of field goals made by the losing team, and so on up to **LPF**).

Note: by convention, "field goals made" (either WFGM or LFGM) refers to the total number of fields goals made by a team, a combination of both two-point field goals and three-point field goals. And "three point field goals made" (either WFGM3 or LFGM3) is just the three-point fields goals made, of course. So if you want to know specifically about two-point field goals, you have to subtract one from the other (e.g., WFGM - WFGM3). And the total number of points scored is most simply expressed as (2*FGM) + FGM3 + FTM.

## Data Section 3 - Geography

**This section provides city locations of all regular season, conference tournament, and NCAA® tournament games since the 2010 season**

## Data Section 4 - Public Rankings

**This section provides weekly team rankings (men's teams only) for dozens of top rating systems - Pomeroy, Sagarin, RPI, ESPN, etc., since the 2003 season.**

## Data Section 5 - Supplements

**This section contains additional supporting information, including coaches, conference affiliations, alternative team name spellings, bracket structure, and game results for NIT and other postseason tournaments.**



In [21]:
######################
# 1. LOADING THE DATA #
######################

import itertools
import pandas as pd
import numpy as np
from sklearn.metrics import brier_score_loss, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


def reduce_mem_usage(df):
    """
    Reduce DataFrame memory usage by converting columns to more efficient dtypes.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        
        # Skip conversion for categorical/string columns
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # If column is integer
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                # If column is float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Convert object columns to categorical when feasible
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    
    return df


def load_optimized_csv(file_path):
    """
    Attempts to load a CSV with a memory-optimized approach.
    1) Tries UTF-8 encoding, falls back to ISO-8859-1 if error.
    2) Uses low_memory=False to improve parsing of large files.
    3) Runs reduce_mem_usage on the resulting DataFrame.
    """
    try:
        df = pd.read_csv(file_path, low_memory=False, encoding='utf-8')
    except UnicodeDecodeError:
        # Fallback to ISO-8859-1 if UTF-8 fails
        df = pd.read_csv(file_path, low_memory=False, encoding='ISO-8859-1')
    
    df = reduce_mem_usage(df)
    return df


# Dictionary of all competition files to load
dataset_files = {
    "m_teams": "MTeams.csv",
    "w_teams": "WTeams.csv",
    "m_seasons": "MSeasons.csv",
    "w_seasons": "WSeasons.csv",
    "m_tourney_seeds": "MNCAATourneySeeds.csv",
    "w_tourney_seeds": "WNCAATourneySeeds.csv",
    "m_regular_season_compact": "MRegularSeasonCompactResults.csv",
    "w_regular_season_compact": "WRegularSeasonCompactResults.csv",
    "m_tourney_compact": "MNCAATourneyCompactResults.csv",
    "w_tourney_compact": "WNCAATourneyCompactResults.csv",
    "m_regular_season_detailed": "MRegularSeasonDetailedResults.csv",
    "w_regular_season_detailed": "WRegularSeasonDetailedResults.csv",
    "m_tourney_detailed": "MNCAATourneyDetailedResults.csv",
    "w_tourney_detailed": "WNCAATourneyDetailedResults.csv",
    "cities": "Cities.csv",
    "m_game_cities": "MGameCities.csv",
    "w_game_cities": "WGameCities.csv",
    "massey_ordinals": "MMasseyOrdinals.csv",
    "m_team_coaches": "MTeamCoaches.csv",
    "conferences": "Conferences.csv",
    "m_team_conferences": "MTeamConferences.csv",
    "w_team_conferences": "WTeamConferences.csv",
    "m_conf_tourney_games": "MConferenceTourneyGames.csv",
    "w_conf_tourney_games": "WConferenceTourneyGames.csv",
    "m_secondary_tourney_teams": "MSecondaryTourneyTeams.csv",
    "w_secondary_tourney_teams": "WSecondaryTourneyTeams.csv",
    "m_secondary_tourney_results": "MSecondaryTourneyCompactResults.csv",
    "w_secondary_tourney_results": "WSecondaryTourneyCompactResults.csv",
    "m_team_spellings": "MTeamSpellings.csv",
    "w_team_spellings": "WTeamSpellings.csv",
    "m_tourney_slots": "MNCAATourneySlots.csv",
    "w_tourney_slots": "WNCAATourneySlots.csv",
    "m_seed_round_slots": "MNCAATourneySeedRoundSlots.csv"
}

# -------------- LOAD ALL DATAFRAMES --------------
base_path = "/kaggle/input/march-machine-learning-mania-2025"
dataframes = {}

print("🔹 Starting to load all CSV files with memory optimization...\n")
for key, file_name in dataset_files.items():
    full_path = f"{base_path}/{file_name}"
    dataframes[key] = load_optimized_csv(full_path)
print("\n✅ All files loaded into the `dataframes` dictionary.\n")

# -------------- UNPACK INTO VARIABLES --------------
(
    m_teams, w_teams, m_seasons, w_seasons, m_tourney_seeds, w_tourney_seeds,
    m_regular_season_compact, w_regular_season_compact, m_tourney_compact, w_tourney_compact,
    m_regular_season_detailed, w_regular_season_detailed, m_tourney_detailed, w_tourney_detailed,
    cities, m_game_cities, w_game_cities, massey_ordinals, m_team_coaches, conferences,
    m_team_conferences, w_team_conferences, m_conf_tourney_games, w_conf_tourney_games,
    m_secondary_tourney_teams, w_secondary_tourney_teams, m_secondary_tourney_results,
    w_secondary_tourney_results, m_team_spellings, w_team_spellings, m_tourney_slots,
    w_tourney_slots, m_seed_round_slots
) = dataframes.values()

print("✅ Data successfully unpacked into individual DataFrames!\n")

🔹 Starting to load all CSV files with memory optimization...


✅ All files loaded into the `dataframes` dictionary.

✅ Data successfully unpacked into individual DataFrames!



## Exploring the Data

In [22]:
###########################
# 2. EXPLORING THE DATA   #
###########################

def summarize_dataframes(df_dict, markdown_file="dataset_summary.txt"):
    """
    Summarize each DataFrame in the provided dictionary, including:
    - # of rows
    - # of columns
    - # missing values
    - # duplicate rows
    - column names
    - first 5 rows (converted to list-of-dicts to avoid display issues)
    
    Saves the summary to a text (markdown) file for easy reference.
    """
    dataset_summary = {}
    
    # Build a summary for each DataFrame
    for name, df in df_dict.items():
        dataset_summary[name] = {
            "Rows": df.shape[0],
            "Columns": df.shape[1],
            "Missing Values": int(df.isnull().sum().sum()),
            "Duplicate Rows": int(df.duplicated().sum()),
            "Column Names": df.columns.tolist(),
            "Sample Rows": df.head().to_dict(orient='records')  # first 5 as list-of-dicts
        }
    
    # Convert summary to a DataFrame and transpose it for readability
    summary_df = pd.DataFrame(dataset_summary).T
    
    # Display in the notebook (if supported)
    print("🔎 Dataset Summaries:\n")
    display(summary_df)  # IPython display; works in most notebooks
    
    # Export summary as Markdown (fallback to .to_string())
    try:
        summary_md = summary_df.to_markdown()
    except ImportError:
        summary_md = summary_df.to_string()
    
    with open(markdown_file, "w", encoding="utf-8") as f:
        f.write(summary_md)
    
    print(f"\n✅ Summary successfully saved to '{markdown_file}'")


# Call the function to summarize all dataframes
summarize_dataframes(dataframes)

🔎 Dataset Summaries:



,Rows,Columns,Missing Values,Duplicate Rows,Column Names,Sample Rows
m_teams,380,4,0,0,"[TeamID, TeamName, FirstD1Season, LastD1Season]","[{'TeamID': 1101, 'TeamName': 'Abilene Chr', '..."
w_teams,378,2,0,0,"[TeamID, TeamName]","[{'TeamID': 3101, 'TeamName': 'Abilene Chr'}, ..."
m_seasons,41,6,0,0,"[Season, DayZero, RegionW, RegionX, RegionY, R...","[{'Season': 1985, 'DayZero': '10/29/1984', 'Re..."
w_seasons,28,6,0,0,"[Season, DayZero, RegionW, RegionX, RegionY, R...","[{'Season': 1998, 'DayZero': '10/27/1997', 'Re..."
m_tourney_seeds,2558,3,0,0,"[Season, Seed, TeamID]","[{'Season': 1985, 'Seed': 'W01', 'TeamID': 120..."
w_tourney_seeds,1676,3,0,0,"[Season, Seed, TeamID]","[{'Season': 1998, 'Seed': 'W01', 'TeamID': 333..."
m_regular_season_compact,191796,8,0,0,"[Season, DayNum, WTeamID, WScore, LTeamID, LSc...","[{'Season': 1985, 'DayNum': 20, 'WTeamID': 122..."
w_regular_season_compact,135948,8,0,0,"[Season, DayNum, WTeamID, WScore, LTeamID, LSc...","[{'Season': 1998, 'DayNum': 18, 'WTeamID': 310..."
m_tourney_compact,2518,8,0,0,"[Season, DayNum, WTeamID, WScore, LTeamID, LSc...","[{'Season': 1985, 'DayNum': 136, 'WTeamID': 11..."
w_tourney_compact,1650,8,0,0,"[Season, DayNum, WTeamID, WScore, LTeamID, LSc...","[{'Season': 1998, 'DayNum': 137, 'WTeamID': 31..."



✅ Summary successfully saved to 'dataset_summary.txt'


## Data Preparation

###  Update Data Preparation to Include ELO

In [23]:
######################
# 3. DATA PREPARATION #
######################

def compute_team_stats(regular_season_df):
    """
    Compute season-level statistics for each team based on regular season results.
    Returns a DataFrame with columns:
      [Season, TeamID, Wins, Losses, TotalGames, WinRatio,
       AvgPointsScored, AvgPointsAllowed, PointMargin]
    
    Parameters
    ----------
    regular_season_df : pd.DataFrame
        Must contain at least these columns:
            - Season (int)
            - WTeamID (int)
            - LTeamID (int)
            - WScore (int)
            - LScore (int)
    
    Returns
    -------
    pd.DataFrame
        A DataFrame with one row per (Season, TeamID).
    """
    # Wins, points scored, points allowed by the "WTeam"
    win_stats = (
        regular_season_df
        .groupby(['Season', 'WTeamID'], as_index=False)
        .agg(
            Wins=('WTeamID', 'count'),
            Points_Scored_Win=('WScore', 'sum'),
            Points_Allowed_Win=('LScore', 'sum')
        )
        .rename(columns={'WTeamID': 'TeamID'})
    )
    
    # Losses, points scored, points allowed by the "LTeam"
    loss_stats = (
        regular_season_df
        .groupby(['Season', 'LTeamID'], as_index=False)
        .agg(
            Losses=('LTeamID', 'count'),
            Points_Scored_Loss=('LScore', 'sum'),
            Points_Allowed_Loss=('WScore', 'sum')
        )
        .rename(columns={'LTeamID': 'TeamID'})
    )
    
    # Merge the two
    team_stats = pd.merge(win_stats, loss_stats, 
                          on=['Season', 'TeamID'], 
                          how='outer').fillna(0)
    
    # Total Games
    team_stats['TotalGames'] = team_stats['Wins'] + team_stats['Losses']
    
    # Win Ratio (handle zero games safely)
    team_stats['WinRatio'] = (
        team_stats['Wins'] / team_stats['TotalGames'].replace({0: np.nan})
    ).fillna(0.0)
    
    # Average Points Scored/Allowed
    team_stats['AvgPointsScored'] = (
        team_stats['Points_Scored_Win'] + team_stats['Points_Scored_Loss']
    ) / team_stats['TotalGames'].replace({0: np.nan})
    
    team_stats['AvgPointsAllowed'] = (
        team_stats['Points_Allowed_Win'] + team_stats['Points_Allowed_Loss']
    ) / team_stats['TotalGames'].replace({0: np.nan})
    
    # Fill potential NaNs (if a team had 0 games, extremely rare but possible in test scenarios)
    team_stats['AvgPointsScored'] = team_stats['AvgPointsScored'].fillna(0.0)
    team_stats['AvgPointsAllowed'] = team_stats['AvgPointsAllowed'].fillna(0.0)
    
    # Point Margin
    team_stats['PointMargin'] = team_stats['AvgPointsScored'] - team_stats['AvgPointsAllowed']
    
    # Drop columns we no longer need
    team_stats.drop(
        ['Points_Scored_Win', 'Points_Scored_Loss', 'Points_Allowed_Win', 'Points_Allowed_Loss'],
        axis=1, inplace=True
    )
    
    # Optimize data types
    # (Adjust dtypes as needed if you see any range/precision issues.)
    team_stats = team_stats.astype({
        'Season': 'int16',
        'TeamID': 'int16',
        'Wins': 'int16',
        'Losses': 'int16',
        'TotalGames': 'int16',
        'WinRatio': 'float16',
        'AvgPointsScored': 'float16',
        'AvgPointsAllowed': 'float16',
        'PointMargin': 'float16'
    })
    
    return team_stats


# --- Compute Stats for Men's and Women's Regular Season ---
m_team_stats = compute_team_stats(m_regular_season_compact)
w_team_stats = compute_team_stats(w_regular_season_compact)

# Display shapes & head
print("Men's Team Stats:")
print(f"Shape: {m_team_stats.shape}")
display(m_team_stats.head())

print("\nWomen's Team Stats:")
print(f"Shape: {w_team_stats.shape}")
display(w_team_stats.head())

# Optionally, save the results to CSV
m_team_stats.to_csv("m_team_stats.csv", index=False)
w_team_stats.to_csv("w_team_stats.csv", index=False)

print("\n✅ Team stats saved to m_team_stats.csv and w_team_stats.csv!")

Men's Team Stats:
Shape: (13388, 9)


,Season,TeamID,Wins,Losses,TotalGames,WinRatio,AvgPointsScored,AvgPointsAllowed,PointMargin
0,1985,1102,5,19,24,0.208374,63.09375,68.8750,-5.792969
1,1985,1103,9,14,23,0.391357,61.03125,64.0625,-3.042969
2,1985,1104,21,9,30,0.700195,68.50000,60.6875,7.800781
3,1985,1106,10,14,24,0.416748,71.62500,75.4375,-3.791016
4,1985,1108,19,6,25,0.759766,83.00000,75.0625,7.960938



Women's Team Stats:
Shape: (9488, 9)


,Season,TeamID,Wins,Losses,TotalGames,WinRatio,AvgPointsScored,AvgPointsAllowed,PointMargin
0,1998,3102,4,20,24,0.166626,57.28125,77.9375,-20.625000
1,1998,3103,11,18,29,0.379395,69.25000,75.1250,-5.863281
2,1998,3104,21,9,30,0.700195,76.56250,63.1250,13.429688
3,1998,3106,6,15,21,0.285645,61.25000,69.1875,-7.953125
4,1998,3108,12,11,23,0.521973,67.81250,66.5000,1.304688



✅ Team stats saved to m_team_stats.csv and w_team_stats.csv!


### Create Matchup Features for Model Training

In [24]:
###########################################
# 4. FEATURE ENGINEERING: MATCHUP FEATURES #
###########################################

def create_matchup_features(tourney_results, team_stats, seeds):
    """
    Create matchup-level features for tournament games. Merges:
      - Tournament results (compact): who won and lost
      - Season-level team stats     : (Wins, Losses, AvgPoints, etc.)
      - Seed information            : numeric seed extracted from 'Seed' column
    
    Parameters
    ----------
    tourney_results : pd.DataFrame
        Must contain columns [Season, DayNum, WTeamID, WScore, LTeamID, LScore].
        'WTeamID' is always the winner; 'LTeamID' always the loser.
    team_stats : pd.DataFrame
        The output of compute_team_stats(), with columns
        [Season, TeamID, Wins, Losses, TotalGames, WinRatio, AvgPointsScored,
         AvgPointsAllowed, PointMargin].
    seeds : pd.DataFrame
        Must contain columns [Season, TeamID, Seed].
        E.g. 'Seed' might look like 'W01' or 'X12a' for a play-in.
    
    Returns
    -------
    pd.DataFrame
        A DataFrame with features describing the difference in stats between
        the winning and losing teams, plus a 'Target' column:
          Target = 1 if the lower TeamID is the winner, else 0.
    """
    # Merge the 'winner' team stats
    df = tourney_results.merge(
        team_stats, 
        left_on=['Season', 'WTeamID'], 
        right_on=['Season', 'TeamID'],
        how='left', 
        suffixes=('_W','_L')  # Temporary suffix, replaced in next merge
    )
    
    # Merge the 'loser' team stats
    df = df.merge(
        team_stats, 
        left_on=['Season', 'LTeamID'], 
        right_on=['Season', 'TeamID'], 
        how='left', 
        suffixes=('_Winner','_Loser')
    )
    
    # Drop the now-redundant 'TeamID_Winner' & 'TeamID_Loser' columns
    df.drop(columns=['TeamID_Winner', 'TeamID_Loser'], inplace=True)
    
    # Prep the seeds data for merging on the winner side
    seeds_w = seeds.rename(columns={'TeamID': 'WTeamID', 'Seed': 'Seed_Winner'})
    df = df.merge(seeds_w, on=['Season', 'WTeamID'], how='left')
    
    # Prep the seeds data for merging on the loser side
    seeds_l = seeds.rename(columns={'TeamID': 'LTeamID', 'Seed': 'Seed_Loser'})
    df = df.merge(seeds_l, on=['Season', 'LTeamID'], how='left')
    
    # Extract numeric portion of the seed (ignore region letter, e.g., 'W', 'X')
    # If seed is missing, default to 17 (larger than typical 16)
    df['Seed_Winner'] = df['Seed_Winner'].str.extract(r'(\d+)').astype(float).fillna(17.0)
    df['Seed_Loser'] = df['Seed_Loser'].str.extract(r'(\d+)').astype(float).fillna(17.0)
    
    # Compute differences in stats (winner minus loser)
    df['WinRatio_Diff']          = df['WinRatio_Winner']          - df['WinRatio_Loser']
    df['PointMargin_Diff']       = df['PointMargin_Winner']       - df['PointMargin_Loser']
    df['AvgPointsScored_Diff']   = df['AvgPointsScored_Winner']   - df['AvgPointsScored_Loser']
    df['AvgPointsAllowed_Diff']  = df['AvgPointsAllowed_Winner']  - df['AvgPointsAllowed_Loser']
    df['Seed_Diff']              = df['Seed_Loser']               - df['Seed_Winner']
    
    # Create Target = 1 if the *lower TeamID* is the winner, else 0
    # This aligns with the competition requirement for final predictions:
    # "We predict the probability that the lower TeamID beats the higher TeamID."
    df['Target'] = (df['WTeamID'] < df['LTeamID']).astype(int)
    
    # Convert columns to smaller dtypes for efficiency
    dtype_map = {
        'Season': 'int16',
        'WTeamID': 'int16',
        'LTeamID': 'int16',
        'Seed_Winner': 'float16',
        'Seed_Loser': 'float16',
        'WinRatio_Diff': 'float16',
        'PointMargin_Diff': 'float16',
        'AvgPointsScored_Diff': 'float16',
        'AvgPointsAllowed_Diff': 'float16',
        'Seed_Diff': 'float16',
        'Target': 'int8'
    }
    for col, dt in dtype_map.items():
        if col in df.columns:
            df[col] = df[col].astype(dt)
    
    return df

# --- Generate Training Data for Men & Women ---
m_training_data = create_matchup_features(m_tourney_compact, m_team_stats, m_tourney_seeds)
w_training_data = create_matchup_features(w_tourney_compact, w_team_stats, w_tourney_seeds)

# Display shapes & sample rows
print("Men's Training Data:")
print(f"Shape: {m_training_data.shape}")
display(m_training_data.head())

print("\nWomen's Training Data:")
print(f"Shape: {w_training_data.shape}")
display(w_training_data.head())

# Optionally, save to CSV
m_training_data.to_csv("m_training_data.csv", index=False)
w_training_data.to_csv("w_training_data.csv", index=False)

print("\n✅ Training data saved to 'm_training_data.csv' and 'w_training_data.csv'!")

Men's Training Data:
Shape: (2518, 30)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Wins_Winner,Losses_Winner,...,AvgPointsAllowed_Loser,PointMargin_Loser,Seed_Winner,Seed_Loser,WinRatio_Diff,PointMargin_Diff,AvgPointsScored_Diff,AvgPointsAllowed_Diff,Seed_Diff,Target
0,1985,136,1116,63,1234,54,N,0,21,12,...,59.28125,10.468750,9.0,8.0,-0.030273,-6.832031,-4.4375,2.40625,-1.0,1
1,1985,136,1120,59,1345,58,N,0,18,11,...,65.31250,3.800781,11.0,6.0,-0.059570,-0.111328,1.2500,1.31250,-5.0,1
2,1985,136,1207,68,1250,43,N,0,25,2,...,70.18750,-4.449219,1.0,16.0,0.546387,20.109375,10.0000,-10.12500,15.0,1
3,1985,136,1229,58,1425,55,N,0,20,7,...,64.62500,3.785156,9.0,8.0,0.062012,2.179688,3.1875,1.00000,-1.0,1
4,1985,136,1242,49,1325,38,N,0,23,7,...,63.00000,4.554688,3.0,14.0,0.025879,1.078125,8.5000,7.37500,11.0,1



Women's Training Data:
Shape: (1650, 30)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Wins_Winner,Losses_Winner,...,AvgPointsAllowed_Loser,PointMargin_Loser,Seed_Winner,Seed_Loser,WinRatio_Diff,PointMargin_Diff,AvgPointsScored_Diff,AvgPointsAllowed_Diff,Seed_Diff,Target
0,1998,137,3104,94,3422,46,H,0,21,9,...,68.3125,6.109375,2.0,15.0,-0.003418,7.320312,2.1250,-5.18750,13.0,1
1,1998,137,3112,75,3365,63,H,0,21,6,...,58.7500,17.828125,3.0,14.0,-0.007812,-5.421875,1.8125,7.18750,11.0,1
2,1998,137,3163,93,3193,52,H,0,30,2,...,64.5625,5.070312,2.0,15.0,0.223145,19.812500,14.6250,-5.18750,13.0,1
3,1998,137,3198,59,3266,45,H,0,28,1,...,63.8125,9.539062,7.0,10.0,0.196289,15.570312,7.1875,-8.34375,3.0,1
4,1998,137,3203,74,3208,72,A,0,17,9,...,72.0625,4.730469,10.0,7.0,0.000000,4.925781,-6.8125,-11.78125,-3.0,1



✅ Training data saved to 'm_training_data.csv' and 'w_training_data.csv'!


## Computing ELO Ratings and Submission

In [31]:
##############################################################
#  COMPUTE ELO RATINGS FOR MEN & WOMEN (REGULAR SEASON)   #
##############################################################

def compute_elo_ratings(
    regular_season_df, 
    start_elo=1500, 
    k_factor=20, 
    season_scoped=True
):
    """
    Compute ELO ratings for each team using a simple ELO model.
    
    Parameters
    ----------
    regular_season_df : pd.DataFrame
        Must have columns [Season, DayNum, WTeamID, LTeamID, WScore, LScore].
    start_elo : int
        The initial ELO rating for all teams at the start of their first season (or each season).
    k_factor : int or float
        How fast ELO ratings adjust after each game.
    season_scoped : bool
        If True, each season starts fresh at `start_elo` for every team.
        If False, ELOs carry over from one season to the next.
    
    Returns
    -------
    dict
        - If season_scoped=True: { (Season, TeamID): ELO } 
        - If season_scoped=False: { TeamID: ELO } 
    """
    # Sort all games in chronological order
    regular_season_df = regular_season_df.sort_values(by=['Season', 'DayNum']).reset_index(drop=True)
    
    elo_dict = {}
    
    for idx, row in regular_season_df.iterrows():
        season = int(row['Season'])
        w_tid = int(row['WTeamID'])
        l_tid = int(row['LTeamID'])
        
        # Decide the dictionary key
        if season_scoped:
            w_key = (season, w_tid)
            l_key = (season, l_tid)
        else:
            w_key = w_tid
            l_key = l_tid
        
        # Initialize to start_elo if not present
        if w_key not in elo_dict:
            elo_dict[w_key] = start_elo
        if l_key not in elo_dict:
            elo_dict[l_key] = start_elo
        
        w_elo = elo_dict[w_key]
        l_elo = elo_dict[l_key]
        
        # Expected score for the winner
        exp_w = 1.0 / (1.0 + 10 ** ((l_elo - w_elo) / 400.0))
        
        # Actual outcome: winner=1, loser=0
        new_w_elo = w_elo + k_factor * (1.0 - exp_w)
        new_l_elo = l_elo + k_factor * (0.0 - (1.0 - exp_w))  # or l_elo + k_factor * (0 - (1 - exp_w))
        
        elo_dict[w_key] = new_w_elo
        elo_dict[l_key] = new_l_elo
    
    return elo_dict


print("🔹 Computing ELO for Men's data...")
m_elo_dict = compute_elo_ratings(
    m_regular_season_compact, 
    start_elo=1500, 
    k_factor=20, 
    season_scoped=True
)
print("✅ Men's ELO ratings computed!")

print("\n🔹 Computing ELO for Women's data...")
w_elo_dict = compute_elo_ratings(
    w_regular_season_compact, 
    start_elo=1500, 
    k_factor=20, 
    season_scoped=True
)
print("✅ Women's ELO ratings computed!")


##############################################################
#  EXTRACT FINAL 2025 ELO INTO DATAFRAMES FOR MEN/WOMEN  #
##############################################################

def extract_final_season_elo(elo_dict, season=2025):
    """
    Convert a { (season, TeamID): ELO } dictionary (if season_scoped=True)
    into a DataFrame with columns [TeamID, Final_ELO].
    """
    records = []
    for key, elo in elo_dict.items():
        # key could be (s, teamID)
        if isinstance(key, tuple) and len(key) == 2:
            s, t = key
            if s == season:
                records.append((t, elo))
        else:
            # If you used season_scoped=False, you might need a different filter
            # but for this competition, we typically do season_scoped=True
            pass
    
    df = pd.DataFrame(records, columns=["TeamID", "Final_ELO"])
    return df

m_teams_2025_elo = extract_final_season_elo(m_elo_dict, 2025)
w_teams_2025_elo = extract_final_season_elo(w_elo_dict, 2025)

print(f"\nMen's ELO in 2025: {m_teams_2025_elo.shape[0]} teams")
display(m_teams_2025_elo.head())

print(f"Women's ELO in 2025: {w_teams_2025_elo.shape[0]} teams")
display(w_teams_2025_elo.head())

# Optionally, save these to CSV for later reuse
m_teams_2025_elo.to_csv("m_teams_2025_elo.csv", index=False)
w_teams_2025_elo.to_csv("w_teams_2025_elo.csv", index=False)


###############################################################
#  GET ALL 2025 DIVISION-I TEAMS & CREATE MATCHUPS         #
###############################################################

def get_d1_teams_for_2025(m_teams, w_teams):
    """
    Filter men’s and women’s teams to those active in 2025.
    m_teams has 'FirstD1Season' and 'LastD1Season' columns.
    w_teams typically doesn't, so we assume all women's teams are valid
    (or add logic if you have it).
    
    Returns: (m_teams_2025_d1, w_teams_2025_d1)
    """
    # For men: filter where 2025 is in [FirstD1Season, LastD1Season]
    m_teams_2025_d1 = m_teams[
        (m_teams["FirstD1Season"] <= 2025) & (m_teams["LastD1Season"] >= 2025)
    ].copy()
    
    # For women: assume all are D1, or apply your own filter
    w_teams_2025_d1 = w_teams.copy()
    
    return m_teams_2025_d1, w_teams_2025_d1

m_teams_2025_d1, w_teams_2025_d1 = get_d1_teams_for_2025(m_teams, w_teams)

print(f"\nMen’s D1 teams (2025) from MTeams.csv: {len(m_teams_2025_d1)}")
print(f"Women’s D1 teams (2025) from WTeams.csv: {len(w_teams_2025_d1)}")


def generate_all_matchups(team_ids):
    """
    Generate all pairwise combinations (TeamID1, TeamID2).
    itertools.combinations(...) yields (t1, t2) with t1 < t2 by default.
    """
    sorted_ids = sorted(team_ids)
    return list(itertools.combinations(sorted_ids, 2))

m_matchups = generate_all_matchups(m_teams_2025_d1["TeamID"].unique())
w_matchups = generate_all_matchups(w_teams_2025_d1["TeamID"].unique())
print(f"Men’s possible matchups: {len(m_matchups)}")
print(f"Women’s possible matchups: {len(w_matchups)}")


##########################################################
#  BUILD SUBMISSION.CSV USING ELO-BASED PREDICTIONS   #
##########################################################

def matchup_probability_ELO(t1, t2, elo_df):
    """
    Return probability that the lower ID (t1) beats the higher (t2)
    using standard ELO formula.
    elo_df must have [TeamID, Final_ELO].
    """
    e1_arr = elo_df.loc[elo_df["TeamID"] == t1, "Final_ELO"]
    e2_arr = elo_df.loc[elo_df["TeamID"] == t2, "Final_ELO"]
    
    e1 = e1_arr.values[0] if not e1_arr.empty else 1500
    e2 = e2_arr.values[0] if not e2_arr.empty else 1500
    
    elo_diff = e1 - e2
    return 1.0 / (1.0 + 10.0 ** (-elo_diff / 400.0))

def build_submission_rows(matchups, elo_df, season=2025):
    """
    For each (t1, t2) in matchups, produce:
      ID   = f"{season}_{t1}_{t2}"
      Pred = Probability that t1 wins (where t1 < t2)
    Returns a list of (ID, Pred) tuples.
    """
    rows = []
    for (t1, t2) in matchups:
        p = matchup_probability_ELO(t1, t2, elo_df)
        row_id = f"{season}_{t1}_{t2}"
        rows.append((row_id, p))
    return rows

# -- Build men’s submission rows
print("\n🔹 Building men’s submission rows based on ELO...")
m_rows = build_submission_rows(m_matchups, m_teams_2025_elo, season=2025)

# -- Build women’s submission rows
print("🔹 Building women’s submission rows based on ELO...")
w_rows = build_submission_rows(w_matchups, w_teams_2025_elo, season=2025)

# Combine & create final submission DataFrame
submission_data = pd.DataFrame(m_rows + w_rows, columns=["ID", "Pred"])
print(f"Total submission rows (men + women): {len(submission_data)}")

# Save to CSV
submission_data.to_csv("submission.csv", index=False)
print("\n✅ Final submission saved to 'submission.csv'!")
print("Upload 'submission.csv' on Kaggle for the official 2025 Brier Score.")

🔹 Computing ELO for Men's data...
✅ Men's ELO ratings computed!

🔹 Computing ELO for Women's data...
✅ Women's ELO ratings computed!

Men's ELO in 2025: 364 teams


,TeamID,Final_ELO
0,1104,1651.851034
1,1421,1562.524708
2,1112,1603.610290
3,1145,1345.127407
4,1117,1564.621480


Women's ELO in 2025: 362 teams


,TeamID,Final_ELO
0,3103,1423.212546
1,3209,1508.827194
2,3104,1626.473375
3,3309,1351.894124
4,3112,1530.841042



Men’s D1 teams (2025) from MTeams.csv: 364
Women’s D1 teams (2025) from WTeams.csv: 378
Men’s possible matchups: 66066
Women’s possible matchups: 71253

🔹 Building men’s submission rows based on ELO...
🔹 Building women’s submission rows based on ELO...
Total submission rows (men + women): 137319

✅ Final submission saved to 'submission.csv'!
Upload 'submission.csv' on Kaggle for the official 2025 Brier Score.
